In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

!pip3 install pybind11

In [ ]:
!make

In [ ]:
import sys
sys.path.append('./python')

In [ ]:
!python3 -m pytest -l -v -k "pad_forward"